In [1]:
# !pip install hmmlearn

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from sklearn.metrics import confusion_matrix
from itertools import permutations
import warnings
warnings.filterwarnings("ignore")

In [3]:
b_health_class = pd.read_csv('B.HEALTH classification.csv')
b_health_comp = pd.read_csv('B.Health components.csv')
sleep_qual_class = pd.read_csv('SleepQual classification.csv')
sleep_qual_comp = pd.read_csv('SleepQual components.csv')

b_health_class['subject'] = b_health_class['subject'].ffill()
b_health_comp['subject'] = b_health_comp['subject'].ffill()
sleep_qual_class['subject'] = sleep_qual_class['subject'].ffill()
sleep_qual_comp['subject'] = sleep_qual_comp['subject'].ffill()

merged = b_health_class.merge(b_health_comp, on=['subject', 'day'], how='inner', suffixes=('', '_bhc'))
merged = merged.merge(sleep_qual_class, on=['subject', 'day'], how='inner', suffixes=('', '_sqc'))
merged = merged.merge(sleep_qual_comp, on=['subject', 'day'], how='inner', suffixes=('', '_sqcomp'))

print(f"Merged data shape: {merged.shape}")

Merged data shape: (182, 51)


In [4]:
merged

,subject,day,step count,distance travelled (km),calories burnt (kcal),acive time (minutes),actual sleep duration (minutes),sleep onset latency (minutes),duration in bed (minutes),efficiency (%),...,deep sleep percentage (%)_sqc,REM percentage(%),label_sqc,comp 1,comp 2,comp 3,comp 4,comp 5,comp 6,SleepQual
0,1.0,1,3016,2.28,2151,32,455,29.0,518,88.000000,...,12.548263,24.000000,0,1,0,0,3,0,0,4
1,1.0,2,1790,1.39,2129,19,320,10.0,360,89.000000,...,33.300000,13.800000,0,0,2,0,0,0,1,2
2,1.0,3,2864,2.07,2032,29,453,23.0,504,90.000000,...,11.111111,26.000000,0,1,0,0,3,0,1,4
3,1.0,4,1419,1.80,1713,15,449,32.5,518,87.000000,...,10.424710,27.000000,1,2,0,0,3,0,1,5
4,1.0,5,1698,1.30,1725,18,358,21.0,403,89.000000,...,13.895782,23.000000,1,1,2,0,3,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,24.0,3,6752,4.59,222,67,570,2.5,575,99.130435,...,17.391304,20.434783,0,0,0,0,0,0,0,0
178,24.0,4,6207,3.89,201,61,535,5.5,546,97.985348,...,10.073260,21.978022,0,0,0,0,0,0,0,0
179,24.0,5,6202,4.21,204,67,480,0.0,480,100.000000,...,4.166667,23.958333,0,0,0,0,0,1,0,1
180,24.0,6,1011,0.23,19,21,300,32.5,365,82.191781,...,8.767123,18.356164,2,2,3,1,3,1,1,11


#### Observed Features
- `total phone usage / day (minutes)`: total phone usage in minutes
- `night time phone usage / day (minutes)`: phone usage in minutes during the night
- `awake percentage (%)`: percentage of time in bed spent awake
- `sleep onset latency (minutes)`: time it takes to fall asleep in minutes
- `actual sleep duration (minutes)`: actual sleep duration in minutes


In [5]:
feature_cols = [
    'total phone usage / day (minutes)',
    'night time phone usage / day (minutes)',
    'awake percentage (%)',
    'sleep onset latency (minutes)',
    'actual sleep duration (minutes)'
]

sleep = merged[feature_cols + ['SleepQual']].copy()

# the lower the score, the better the sleep quality
def sleep_level(x):
    if x <= 3: return 0
    elif x <= 6: return 1
    else: return 2

sleep['Sleep_Quality_State'] = sleep['SleepQual'].apply(sleep_level)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(sleep[feature_cols])

states = sleep['Sleep_Quality_State'].values
n_states = len(np.unique(states))

print(f"Number of states: {n_states}")
print(f"State distribution: {np.bincount(states)}")
print(f"X_scaled shape: {X_scaled.shape}")

Number of states: 3
State distribution: [95 58 29]
X_scaled shape: (182, 5)


In [6]:
# Initialize HMM
model = GaussianHMM(n_components=n_states, covariance_type="full", n_iter=200, init_params="")

# Uniform start prob
model.startprob_ = np.ones(n_states) / n_states
print(f"Start prob: {model.startprob_}")

# Transition matrix
A = np.random.rand(n_states, n_states)
A = A / A.sum(axis=1, keepdims=True)
model.transmat_ = A
print(f"Transition matrix:\n{model.transmat_}")

# Emission matrix
means = []
covs = []

for s in np.unique(states):
    X_s = X_scaled[states == s]
    means.append(X_s.mean(axis=0))
    covs.append(np.cov(X_s.T) + 1e-3 * np.eye(X_s.shape[1]))

model.means_ = np.array(means)
model.covars_ = np.array(covs)

print(f"Means shape: {model.means_.shape}")

Start prob: [0.33333333 0.33333333 0.33333333]
Transition matrix:
[[0.53752137 0.34953725 0.11294139]
 [0.47097332 0.45380957 0.07521712]
 [0.55358417 0.18572649 0.26068935]]
Means shape: (3, 5)


In [7]:
model.fit(X_scaled)

GaussianHMM(covariance_type='full', init_params='', n_components=3, n_iter=200)

In [8]:
hidden_states = model.predict(X_scaled)
sleep['Predicted_Sleep_Quality'] = hidden_states

In [9]:
# Direct accuracy
accuracy = np.mean(hidden_states == states)
print(f"\nViterbi reconstruction accuracy: {accuracy}")

# Confusion Matrix
cm = confusion_matrix(states, hidden_states)
print(f"\nConfusion Matrix:\n{cm}")


Viterbi reconstruction accuracy: 0.7692307692307693

Confusion Matrix:
[[78 16  1]
 [11 36 11]
 [ 0  3 26]]
